# Лекция 6. Горизонтальное масштабирование в MongoDB, репликация

В MongoDB реализованы два варианта репликации: **главный-подчиненный и наборы реплик**. В обоих случаях единственный первичный узел выполняет все операции записи, после чего вторичные узлы считывают описания этих операций и асинхронно применяют их у себя.

Вариант **наборы реплик**  - это усовершенствованная технология репликации
типа главный-подчиненный, рекомендуемая для MongoDB.

Репликация типа **главный-подчиненный** была исходной моделью репликации в MongoDB. Она легко конфигурируется и способна поддержать любое количество подчиненных узлов. Но теперь этот
тип репликации не рекомендуется использовать в производственных системах. 

Тому есть две причины. Во-первых, отработка отказа должна производиться целиком вручную. Если главный узел выходит из строя, то администратор должен остановить подчиненный узел
и перезапустить его в режиме главного. Затем необходимо переконфигурировать приложение, так чтобы оно работало с новым главным узлом. Во-вторых, восстановление затруднено. Поскольку журнал операций существует только на главном узле, в случае отказа приходится создавать новый журнал операций на новом главном узле. Это означает, что все прочие узлы должны будут заново синхронизироваться с новым главным.

Начнем с простого набора реплик.

<img src="ris1.jpg">

Минимальная рекомендуемая конфигурация набора реплик включает три узла. Два из них – полноценные экземпляры mongod. Каждый
может выступать в роли первичного узла набора реплик, и на обоих могут храниться полные копии данных. Третий узел играет роль арбитра; он не реплицирует данные, а выполняет функции нейтрального наблюдателя. Как следует из названия, этот узел занимается арбитражем: в случае отказа арбитр помогает выбрать новый первичный узел.

Для каждого узла создадим отдельный каталог:

> mkdir ~/data/node1

> mkdir ~/data/node2

> mkdir ~/data/arbiter

Затем запустим для каждого узла отдельный экземпляр mongod . Лучше запускать разные mongod в разных консольных окнах:

> mongod --replSet myapp --dbpath ~/data/node1 --port 40000

> mongod --replSet myapp --dbpath ~/data/node2 --port 40001

> mongod --replSet myapp --dbpath ~/data/arbiter --port 40002

Далее необходимо сконфигурировать набор реплик. Для этого сначала подключитесь к одному из запущенных экземпляров mongod, но не к арбитру. 

> mongo --port 40000

После подключения необходимо выполнить команду 

> rs.initiate()

{
	"info2" : "no configuration specified. Using a default configuration for the set",
	"me" : "localhost:40000",
	"ok" : 1,
	"operationTime" : Timestamp(1652997864, 1),
	"$clusterTime" : {
		"clusterTime" : Timestamp(1652997864, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	}
}

Примерно через минуту мы получим набор реплик.

Чтобы добавить две других, нужно выполнить команду rs.add():

> rs.add("localhost:40001")

> rs.addArb('localhost:40002')

Для второго узла мы задали параметр arbiterOnly , то есть сконфигурировали его для работы в качестве арбитра. Через некоторое время все ноды должны перейти в рабочий режим. Чтобы получить краткие сведения о состоянии набора реплик, можно выполнить команду db.isMaster():

> db.isMaster()

Более подробную информацию о системе дает метод rs.status():

> rs.status()

Если база данных MongoDB не слишком велика, то набор реплик должен перейти в рабочий режим в течение 30 секунд. На протяжении этого времени поле stateStr в каждом узле будет сначала равно RECOVERING , а потом – PRIMARY , SECONDARY или ARBITER.

Пробуем проверить, как работает репликация:

Подключаемся к мастеру:

> mongo --port 40000

> use test

> db.rebus.save({name:'Egor', birthday:'10.04.2008'})

Подключимся из другого окна терминала к вторичной базе:

> mongo --port 40001

Чтобы разрешить считывание из вторичных узлов, нужно сообщить об этом к подключенному вторичному узлу, с помощью команды:

> rs.slaveOk()

После чего выберем базу и посмотрим, что в ней:

> use test

> db.rebus.find()

Если все произошло, как показано, значит, набор реплик успешно сконфигурирован.

Теперь проверим автоматическую отработку отказа.

Имитировать отказ сети проблематично, поэтому пойдем по более простому пути – остановим сам узел. Можно остановить вторичный
узел, но тогда мы всего лишь прекратим репликацию, а остальные узлы сохранят текущее состояние. Если же вы хотите наблюдать,
как изменяется состояние системы, то нужно остановить первичный узел. 

Воспользуйтесь стандартной комбинацией клавиш Ctrl+C  на вкладке консоля, где запущен мастер. Можно также подключиться к первичному узлу и выполнить в оболочке команду db.shutdownServer().

После того как первичный узел остановлен, вторичный обнаруживает отсутствие периодических тактовых сигналов от него. Тогда 
вторичный узел выбирает себя на роль первичного. Такой выбор возможен, потому что большинство исходных узлов (арбитр и исходный вторичный) могут прозванивать друг друга.

Если сейчас подключиться к новому первичному узлу и проверить состояние набора реплик, то будет видно, что старый первичный
узел недоступен:

> rs.status()

После отработки отказа в наборе реплик осталось всего два узла.

Можно попробовать восстановить узел на порту 40000 и присоединить его к репликации. В этом случае он присоединится уже как Secondary. 

После остановки всех серверов, чтобы заново поднять весь кластер, необходимо их все заново запустить.